# SCRAPING DE JUMIA AVEC SELENIUM

L'objectif de cet exercice est de collecter les produits disponibles sur JUMIA et réaliser une analyse comparative des prix de ces différents produits

In [60]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.common.keys import Keys
import os
import time
import pandas as pd

## URL

In [3]:
url = "https://jumia.ci"

In [83]:
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)
driver.find_element(By.CSS_SELECTOR, '.popup').find_element(By.CSS_SELECTOR, 'button').click()

## Rechercher iphone dans barre de recherche

In [84]:
recherche = driver.find_element(By.ID, "fi-q")
recherche.send_keys("iphone")
recherche.send_keys(Keys.ENTER)

## Informations sur l'iphone

In [85]:
produit = driver.find_element(By.CSS_SELECTOR, '.prd .core')
lien_details = produit.get_attribute('href')
lien_details
info = produit.find_element(By.CSS_SELECTOR, ".info")

boutique = info.find_element(By.CSS_SELECTOR, ".bdg").text
nom = info.find_element(By.CSS_SELECTOR, ".name").text
prix = info.find_element(By.CSS_SELECTOR, ".prc").text
ancien_prix = info.find_element(By.CSS_SELECTOR, ".s-prc-w").text.split('\n')[0]
ancien_prix
note = info.find_element(By.CSS_SELECTOR, ".stars").text.split()[0]
note

nombre_avis = info.find_element(By.CSS_SELECTOR, ".rev").text.split('\n')[1].replace('(', '').replace(')','') 

## Informations sur l'ensemble des iphones

In [90]:

produits = driver.find_elements(By.CSS_SELECTOR, '.prd .core')

res = []
for prod in produits:
    lien_details = prod.get_attribute('href')
    lien_details
    info = prod.find_element(By.CSS_SELECTOR, ".info")
    nom = info.find_element(By.CSS_SELECTOR, ".name").text
    prix = info.find_element(By.CSS_SELECTOR, ".prc").text

    try:
        boutique_officielle = True if info.find_element(By.CSS_SELECTOR, ".bdg._mall") else False
    except:
        boutique_officielle = False
        
    try:
        ancien_prix = info.find_element(By.CSS_SELECTOR, ".s-prc-w").text.split('\n')[0]
    except:
        ancien_prix = None
        
    try:
        note = info.find_element(By.CSS_SELECTOR, ".stars")
    except:
        note = None
        
    try:
        nombre_avis = info.find_element(By.CSS_SELECTOR, ".rev").text.split('\n')[1].replace('(', '').replace(')','') 
    except:
        nombre_avis = 0
        
    resultat = {
        "Nom": nom,
        "Prix Actuel": prix,
        "Ancien Prix": ancien_prix,
        "Lien": lien_details,
        "Boutique Officielle": boutique_officielle,
        "Note": note.text.split()[0] if note else None,
        "Avis": nombre_avis
    }
    
    res.append(resultat)
res

[{'Nom': 'Renewed IPhone 6 64GO (Reconditionné)',
  'Prix Actuel': '39,900 FCFA',
  'Ancien Prix': '99,000 FCFA',
  'Lien': 'https://www.jumia.ci/renewed-iphone-6-64go-reconditionne-26623309.html',
  'Boutique Officielle': True,
  'Note': '3.5',
  'Avis': '113'},
 {'Nom': 'Renewed IPhone 6 - RAM 1Go - 64 Go ROM - Remis à Neuf -Or',
  'Prix Actuel': '39,900 FCFA',
  'Ancien Prix': '104,510 FCFA',
  'Lien': 'https://www.jumia.ci/renewed-iphone-6-ram-1go-64-go-rom-remis-a-neuf-or-29380903.html',
  'Boutique Officielle': False,
  'Note': None,
  'Avis': 0},
 {'Nom': 'Apple IPhone 13 - 128Go - Noire',
  'Prix Actuel': '419,000 FCFA',
  'Ancien Prix': '1,000,000 FCFA',
  'Lien': 'https://www.jumia.ci/apple-iphone-13-128go-noire-19732039.html',
  'Boutique Officielle': True,
  'Note': '5',
  'Avis': '1'},
 {'Nom': 'Apple IPHONE 15-128Go Noir',
  'Prix Actuel': '589,000 FCFA',
  'Ancien Prix': '1,000,000 FCFA',
  'Lien': 'https://www.jumia.ci/apple-iphone-15-128go-noir-27877429.html',
  'Bouti

In [92]:
df = pd.DataFrame(res)
df

,Nom,Prix Actuel,Ancien Prix,Lien,Boutique Officielle,Note,Avis
0,Renewed IPhone 6 64GO (Reconditionné),"39,900 FCFA","99,000 FCFA",https://www.jumia.ci/renewed-iphone-6-64go-rec...,True,3.5,113
1,Renewed IPhone 6 - RAM 1Go - 64 Go ROM - Remis...,"39,900 FCFA","104,510 FCFA",https://www.jumia.ci/renewed-iphone-6-ram-1go-...,False,None,0
2,Apple IPhone 13 - 128Go - Noire,"419,000 FCFA","1,000,000 FCFA",https://www.jumia.ci/apple-iphone-13-128go-noi...,True,5,1
3,Apple IPHONE 15-128Go Noir,"589,000 FCFA","1,000,000 FCFA",https://www.jumia.ci/apple-iphone-15-128go-noi...,True,None,0
4,Chargeur iPhone Original,"3,000 FCFA","6,000 FCFA",https://www.jumia.ci/generic-chargeur-iphone-o...,False,3.6,5
5,Apple IPhone 13 ProMax-256Go Graphite,"909,000 FCFA","1,500,000 FCFA",https://www.jumia.ci/apple-iphone-13-promax-25...,True,None,0
6,Apple IPhone 15Pro-256Go Titane Blanc,"809,000 FCFA","1,500,000 FCFA",https://www.jumia.ci/apple-iphone-15pro-256go-...,True,None,0
7,"Apple IPad Air 10.9"" WI-FI + Cellular","1,000,000 FCFA",None,https://www.jumia.ci/apple-ipad-air-10.9-wi-fi...,True,None,0
8,Apple IPHONE 15Pro-256Go Titane Naturel,"809,000 FCFA","1,500,000 FCFA",https://www.jumia.ci/apple-iphone-15pro-256go-...,True,None,0
9,"Apple IPHONE 15 PRO - 6.1"" Super Retina XDR OL...","649,800 FCFA",None,https://www.jumia.ci/apple-iphone-15-pro-6.1-s...,True,None,0


In [95]:
driver.find_element(By.CSS_SELECTOR, '[aria-label="Page suivante"]').click()